## Введение

В своём проекте с работала с датасетом, который содержит информацию о **государственных заказчиках города Москвы**, с **Портала открытых данных Правительства Москвы**: https://data.mos.ru/opendata?sorting=byNameCatalog-ASC&activeStatus=true.

Датасет (актуализирован: 10.10.2024) содержит следующую инфорцию о различных учреждениях Москвы:
- ID: Уникальный идентификатор учреждения
- Name: Название учреждения
- MainDisposerOfBudget: Основной распорядитель бюджета
- LegalAddress: Юридический адрес
- OKOPF_Code: Код по Общероссийскому классу организационно-правовых форм
- INN: Идентификационный номер налогоплательщика
- KPP: Код причины постановки на учет
- OGRN: Основной государственный регистрационный номер
- PublicPhone: Телефон для связи
- Email: Электронная почта для связи
- global_id: Глобальный идентификатор

## Вывод данных

In [107]:
import xml.etree.ElementTree as ET

xml_file_path = 'data/datamos.xml'
tree = ET.parse(xml_file_path)
root_element = tree.getroot()
for element in root_element.iter():
    text_value = element.text.strip() if element.text and element.text.strip() else ''
    print(f"{element.tag}: {text_value}")

catalog: 
array: 
item: 
ID: 4612682
Name: Государственное автономное учреждение города Москвы «Московское агентство реализации общественных проектов»
MainDisposerOfBudget: Департамент территориальных органов исполнительной власти города Москвы
LegalAddress: 107392, город Москва, улица Малая Черкизовская, дом 22
OKOPF_Code: 75201
INN: 7718550691
KPP: 771801001
OGRN: 1057747228703
PublicPhone: 
PublicPhone: (499) 740-37-00
Email: 
Email: zakupki.gau.aop@gmail.com
global_id: 2611886579
item: 
ID: 445
Name: Государственное бюджетное учреждение культуры города Москвы «Московский драматический театр на Малой Бронной»
MainDisposerOfBudget: Департамент культуры города Москвы
LegalAddress: 123104, г. Москва, ул. Малая Бронная, дом 4, строение 2
OKOPF_Code: 75203
INN: 7703001970
KPP: 770301001
OGRN: 1037739587225
PublicPhone: 
PublicPhone: (495) 695-94-35
Email: 
Email: torgi@mbronnaya.ru
global_id: 2611886745
item: 
ID: 1230
Name: Государственное бюджетное учреждение культуры города Москвы «Те

## Вывод отформатированных данных (название учреждения и номер телефона)

Для некоторых видов задач могут потребоваться только некоторые данные из датасета. Например, колл-центру для обзвона учреждений могли бы понадобиться название учреждения и номер телефона. Я написала функцию, которая позволяет отсечь всю ненужную информацию, и выводит только название огранизации и её контактный номер телефона.

In [95]:
# Импорт библиотеки для работы с XML
import xml.etree.ElementTree as ET

# Определение пути к файлу
xml_file_path = 'data/datamos.xml'
# Парсинг XML-файла
tree = ET.parse(xml_file_path)
root_element = tree.getroot()

# Определение функции для извлечения названий организаций и номеров телефонов
def print_org_names_and_phones(element):
    # Поиск массивов и элементов
    for array_element in element.findall('array'):
        for item_element in array_element.findall('item'):
            # Извлечение названия организации
            org_name = item_element.find('Name').text.strip() if item_element.find('Name') is not None else 'N/A'
            # Извлечение номера телефона
            phone_number = item_element.find('PublicPhone/PublicPhone').text.strip() if item_element.find('PublicPhone/PublicPhone') is not None else 'N/A'
            # Форматирование телефонного номера
            if phone_number != 'N/A':
                phone_number = '8' + phone_number
            # Вывод имени и номера телефона
            print(f"{org_name} {phone_number}")
            
# Вывод названия организации и номера телефона
print_org_names_and_phones(root_element)

Государственное автономное учреждение города Москвы «Московское агентство реализации общественных проектов» 8(499) 740-37-00
Государственное бюджетное учреждение культуры города Москвы «Московский драматический театр на Малой Бронной» 8(495) 695-94-35
Государственное бюджетное учреждение культуры города Москвы «Театриум на Серпуховке под руководством Терезы Дуровой» 8(495) 636-27-03
Государственное бюджетное учреждение культуры города Москвы «Театр «Школа драматического искусства» 8(495) 632-92-82
Государственное бюджетное учреждение здравоохранения города Москвы «Челюстно-лицевой госпиталь для ветеранов войн Департамента здравоохранения города Москвы» 8(495) 954-86-46
Государственное бюджетное учреждение культуры города Москвы «Московский театр русской драмы под руководством Михаила Щепенко» 8(495) 915-57-80
Государственное бюджетное учреждение культуры города Москвы «Центр культуры и творчества «Нега» 8(926) 406-46-73
Государственное бюджетное учреждение культуры города Москвы «Театр

## Вывод конкретных данных

Рассмотрим более конкретный кейс. Колл-центру требуется обзвонить только поликлиники, для этой задачи подойдет модуль, который запрашивает ввод данных у пользователя. Пользователь вводит слово "поликлиника", и выводятся все огранизации, содержащие в своём названии это слово, а также их контактный номер телефона. 

In [115]:
import xml.etree.ElementTree as ET

xml_file_path = 'data/datamos.xml'

# Загружаем и парсим XML файл
tree = ET.parse(xml_file_path)
root_element = tree.getroot()

def print_names_and_phones(search_term):
    # Ищем название, которое содержит search_term (например, "школа")
    for array_element in root_element.findall('array'):
        for item_element in array_element.findall('item'):
            name = item_element.find('Name').text if item_element.find('Name') is not None else 'N/A'
            phone_number = item_element.find('PublicPhone/PublicPhone').text if item_element.find('PublicPhone/PublicPhone') is not None else 'N/A'
            
            # Если название содержит искомое слово, выводим его
            if search_term.lower() in name.lower():
                if phone_number != 'N/A':
                    phone_number = '8' + phone_number
                print(name, phone_number)

# Запрашиваем ввод у пользователя
search_term = input("Введите слово(а) для поиска (например, 'поликлиника'): ")
print_names_and_phones(search_term)

Введите слово(а) для поиска (например, 'поликлиника'):  1357


Государственное бюджетное общеобразовательное учреждение города Москвы «Школа № 1357 «На Братиславской» 8(495) 346-45-50


## То же самое, но для электронной почты

Функция упрощает задачу по поиску электронной почты разных учреждений, также как и с номерами телефонов, рядом с названием учреждения отображается адрес электронной почты. Например, в окне ввода я пишу "1357" и сразу вижу электронную почту школы 1357.

In [109]:
import xml.etree.ElementTree as ET

xml_file_path = 'data/datamos.xml'

# Загружаем и парсим XML файл
tree = ET.parse(xml_file_path)
root_element = tree.getroot()

def print_names_and_emails(search_term):
    # Ищем название, которое содержит search_term (например, "школа")
    for array_element in root_element.findall('array'):
        for item_element in array_element.findall('item'):
            name = item_element.find('Name').text if item_element.find('Name') is not None else 'N/A'
            email = item_element.find('Email/Email').text if item_element.find('Email/Email') is not None else 'N/A'
            
            # Если название содержит искомое слово, выводим его
            if search_term.lower() in name.lower():
                print(name, email)

# Запрашиваем ввод у пользователя
search_term = input("Введите слово(а) для поиска (например, 'школа'): ")
print_names_and_emails(search_term)

Введите слово(а) для поиска (например, 'школа'):  1357


Государственное бюджетное общеобразовательное учреждение города Москвы «Школа № 1357 «На Братиславской» ks1357_15@mail.ru


## Вывод адресов электронной почты

Функция подойдет для случаев, когда будет использоваться рассылка какой-то информации на электронную почту. Механизм работы такой же, как и в прошлом модуле, только теперь адреса электронной почты будут выводиться без названий учреждений. Например, в окне ввода я пишу "учреждение культуры", после чего получаю все адреса электронной почты данного вида организаций.

In [113]:
import xml.etree.ElementTree as ET

# Загружаем и парсим XML файл
xml_file_path = 'data/datamos.xml'
tree = ET.parse(xml_file_path)
root_element = tree.getroot()

def extract_names_and_emails(root_element):
    data = []
    for array_element in root_element.findall('array'):
        for item_element in array_element.findall('item'):
            name = item_element.find('Name').text
            email = item_element.find('Email/Email').text
            data.append((name, email))
    return data

search_term = input("Введите слово(а) для поиска (например, 'учреждение культуры'): ")
names_and_emails = extract_names_and_emails(root_element)

# Ищем email по названию
for name, email in names_and_emails:
    if search_term.lower() in name.lower():
        print(email)

Введите слово(а) для поиска (например, 'учреждение культуры'):  поликлиника


atk83@mail.ru
dgp7.kontrakt@mail.ru
dsp46@zdrav.mos.ru
zakupkigp52@mail.ru
sp67@zdrav.mos.ru
stpol35@mail.ru
KleevaSA@zdrav.mos.ru
sp22@zdrav.mos.ru
sp64@zdrav.mos.ru
usolga123@rambler.ru
gp6-torgi@mail.ru
tambovtsev_pk62@mail.ru
gp67@zdrav.mos.ru
gp69@zdrav.mos.ru
dsp38@zdrav.mos.ru
dgp129@inbox.ru
mlpuzvrb@mail.ru
KuzminSN2@zdrav.mos.ru
zakupki210@mail.ru
6500600133@mail.ru
ZharkovaEE@zdrav.mos.ru
dsp59@zdrav.mos.ru
dsp10@mail.ru
sp19@zdrav.mos.ru
dsp39@zdrav.mos.ru
barto84@yandex.ru
tanyit1978@yandex.ru
polka105@yandex.ru
sp49@zdrav.mos.ru
gp220@zdrav.mos.ru
dgp38@zdrav.mos.ru
dgp15@zdrav.mos.ru
SmetaninaSV@zdrav.mos.ru
gp19torgi@mail.ru
MorozovaMO@zdrav.mos.ru
zakupki_gp36@mail.ru
dgp-132@yandex.ru
gp191.zakupki@yandex.ru
148-zakupki@zdrav.mos.ru
gp180zakupki@gmail.com
dsp16@zdrav.mos.ru
gb_dsp6@mail.ru
sp3@zdrav.mos.ru
milla_k@list.ru
gp46@zdrav.mos.ru
TGBtorgi@ya.ru
tyurinaln@zdrav.mos.ru
sp50@zdrav.mos.ru
6081664@mail.ru
KiryanovaNV1@zdrav.mos.ru
dgp145@zdrav.mos.ru
KushakovaLG@

## Итоги проекта

В результате работы у меня получилось четыре функции, каждая из которых способна решать разные задачи при работе с данным датасетом. 
- **Первая функция** выводит в консоль название организации и соответствующий телефонный номер в отформатированном виде.
- **Вторая функция** по запросу пользователя выводит учреждения, относящиеся к одному типу, с их контактными номерами телефона. 
- **Третья функция** по запросу пользователя выводит учреждения, относящиеся к одному типу, с их адресами электронной почты.
- **Четвертая функция** выводит адреса электронной почты учреждений одного типа, заданных пользователем.

Я считаю, что эти функции будут полезны при работе с датасетом, потому что они способны выполнять прикладные повседневные задачи людей, работающих с государственными заказчиками Москвы. Первые три функции сократят время поиска информации за счет удобства отображение информации. Четвертая функция ускорит процесс рассылки писем, так как появится возможность сразу скопировать все адреса за раз.